In [1]:
# Import basic libraries for EDA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from dotenv import load_dotenv
import os
import pingouin as pg
from sklearn.preprocessing import StandardScaler 

In [2]:
# Import dataset
load_dotenv()
dataset_path=os.getenv("DATASET_PATH")
dataset = pd.read_csv(dataset_path)
dataset.head()

,RID,Gender,Ageatscreening,Diagnosis,MMSE0m,HipsASMbaseline,HipsContrastbaseline,HipsCorelationbaseline,HipsVariancebaseline,HipsSumAveragebaseline,...,ERCsContrastbaseline,ERCsCorelationbaseline,ERCsVariancebaseline,ERCsSumAveragebaseline,ERCsSumVariancebaseline,ERCsEntropybaseline,ERCsClusterShadebaseline,ERCs_thicknessbaseline,ERCsVolumebaseline,HipposcampusVolumebaseline
0,3,0,81.3479,3,20.0,NaN,158.27,0.63,218.30,28.37,...,253.10,0.40,208.65,23.39,581.50,NaN,-2568.19,2.31,1176.0,3047.0
1,4,0,67.6904,1,27.0,0.06,147.64,0.55,173.64,44.72,...,220.88,0.48,215.70,33.74,641.90,3.33,4113.01,2.76,1942.0,3449.0
2,5,0,73.8027,0,29.0,0.10,199.66,0.55,222.27,41.18,...,220.37,0.54,232.18,29.18,708.36,2.87,-1388.41,3.18,2044.0,3441.0
3,8,1,84.5945,0,28.0,0.08,184.21,0.53,201.55,43.04,...,198.42,0.54,220.48,26.68,683.50,2.77,-2506.55,2.68,1959.0,2875.0
4,10,1,73.9726,3,24.0,0.11,233.02,0.48,229.88,39.46,...,196.55,0.53,210.63,26.60,645.95,2.72,-1164.02,2.64,1397.0,2700.0


In [3]:
null_dataset = dataset.dropna()
null_dataset

,RID,Gender,Ageatscreening,Diagnosis,MMSE0m,HipsASMbaseline,HipsContrastbaseline,HipsCorelationbaseline,HipsVariancebaseline,HipsSumAveragebaseline,...,ERCsContrastbaseline,ERCsCorelationbaseline,ERCsVariancebaseline,ERCsSumAveragebaseline,ERCsSumVariancebaseline,ERCsEntropybaseline,ERCsClusterShadebaseline,ERCs_thicknessbaseline,ERCsVolumebaseline,HipposcampusVolumebaseline
1,4,0,67.6904,1,27.0,0.06,147.64,0.55,173.64,44.72,...,220.88,0.48,215.70,33.74,641.90,3.33,4113.01,2.76,1942.0,3449.0
2,5,0,73.8027,0,29.0,0.10,199.66,0.55,222.27,41.18,...,220.37,0.54,232.18,29.18,708.36,2.87,-1388.41,3.18,2044.0,3441.0
3,8,1,84.5945,0,28.0,0.08,184.21,0.53,201.55,43.04,...,198.42,0.54,220.48,26.68,683.50,2.77,-2506.55,2.68,1959.0,2875.0
4,10,1,73.9726,3,24.0,0.11,233.02,0.48,229.88,39.46,...,196.55,0.53,210.63,26.60,645.95,2.72,-1164.02,2.64,1397.0,2700.0
5,14,1,78.6137,0,29.0,0.12,188.33,0.49,183.76,39.63,...,241.64,0.44,226.48,35.11,664.29,3.10,8478.33,3.01,1809.0,3292.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,1201,1,75.6411,3,26.0,0.18,173.84,0.59,215.04,33.84,...,281.15,0.37,222.73,27.94,609.75,2.84,-1924.49,1.94,1112.0,2705.0
604,1221,0,71.2712,3,21.0,0.22,174.02,0.58,211.07,30.56,...,224.78,0.47,205.49,27.36,597.20,2.77,1146.39,2.44,1526.0,2913.0
605,1248,1,79.8548,3,23.0,0.19,199.31,0.56,229.63,33.13,...,220.41,0.48,214.97,27.16,639.47,2.79,-919.84,1.99,1215.0,2273.0
606,1253,1,62.7452,3,24.0,0.05,142.05,0.50,147.12,46.21,...,217.38,0.51,220.55,32.54,664.84,3.09,5289.89,3.27,2321.0,3739.0


In [4]:
null_dataset.shape

(445, 24)

### Multiclass

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from feature_engine.imputation import MeanMedianImputer

# let's separate into training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    null_dataset.drop("Diagnosis", axis=1),  
    null_dataset["Diagnosis"],  
    test_size=0.3,  
    random_state=0,  
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((311, 23), (134, 23), (311,), (134,))

In [6]:
sc = StandardScaler()
sc.fit(X_train)

# let's transform the data with the pipeline
X_train_scaled = sc.transform(X_train)
X_test_scaled = sc.transform(X_test)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score, balanced_accuracy_score, make_scorer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

def run_model(model, classifier_name, X_train, y_train):
        
        kf = StratifiedKFold(n_splits=5,
                             shuffle=True,
                             random_state=42,
        )
        
        metrics = {"accuracy": make_scorer(accuracy_score),
                   "balanced_accuracy": make_scorer(balanced_accuracy_score),  
                   "precision": make_scorer(precision_score, average="weighted"), 
                   "recall": make_scorer(recall_score, average="weighted"), 
                   "f1_weighted": make_scorer(f1_score, average="weighted"),
                   "roc_auc_ovr_weighted": make_scorer(roc_auc_score, 
                                                       average="weighted", 
                                                       multi_class="ovr", 
                                                       response_method="predict_proba",),
        }
        
        cross_val_results = cross_validate(model,
                                           X_train,
                                           y_train,
                                           cv=kf,
                                           scoring=metrics,
                                           return_train_score=True,
        )
                
        metric_names = list(metrics.keys())
        mean_train = [round(np.mean(cross_val_results[f"train_{metric}"]), 3) for metric in metric_names]
        std_train = [round(np.std(cross_val_results[f"train_{metric}"]), 3) for metric in metric_names]
        mean_test = [round(np.mean(cross_val_results[f"test_{metric}"]), 3) for metric in metric_names]
        std_test = [round(np.std(cross_val_results[f"test_{metric}"]), 3) for metric in metric_names]
        time = round(np.mean(cross_val_results[f"fit_time"]), 3)
                
        cv_metrics_df = pd.DataFrame({
                "Classifier": classifier_name,
                "Fit Time": time,
                "Metric": metric_names,
                "Mean Train": mean_train,
                "Std Train": std_train,
                "Mean Test": mean_test,
                "Std Test": std_test,
        })
        
        fit_model = model.fit(X_train, y_train)
        
        return fit_model, cv_metrics_df


In [8]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42, class_weight="balanced",
        )
lg = LogisticRegression(multi_class = "auto", solver = "lbfgs", max_iter = 1000, random_state = 42)

svm = SVC(kernel ='rbf', decision_function_shape ='ovr', probability = True, random_state = 42)

dt = DecisionTreeClassifier(criterion ='gini', max_depth = 5, min_samples_split = 10,
                                                  min_samples_leaf = 5, max_features = 'sqrt', random_state = 42)

In [9]:
model_rf, metrics_rf = run_model(rf, "Random Forest", X_train, y_train)
model_dt, metrics_dt = run_model(dt, "Decision Tree", X_train, y_train)
model_lg, metrics_lg = run_model(svm, "Logistic Regression", X_train_scaled, y_train)
model_svm, metrics_svm = run_model(lg, "Support Vector Machine", X_train_scaled, y_train)

c:\Users\steve\Desktop\Notebooks\Thesis-Project\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\steve\Desktop\Notebooks\Thesis-Project\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\steve\Desktop\Notebooks\Thesis-Project\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [10]:
validation_df = pd.concat([metrics_rf, metrics_dt, metrics_lg, metrics_svm])
validation_df_report = validation_df.set_index(["Classifier", "Fit Time", "Metric"])
validation_df_report

Mean Train  Std Train  \
Classifier             Fit Time Metric                                        
Random Forest          0.334    accuracy                   0.933      0.005   
                                balanced_accuracy          0.950      0.006   
                                precision                  0.938      0.005   
                                recall                     0.933      0.005   
                                f1_weighted                0.932      0.006   
                                roc_auc_ovr_weighted       0.991      0.002   
Decision Tree          0.010    accuracy                   0.661      0.013   
                                balanced_accuracy          0.603      0.017   
                                precision                  0.673      0.018   
                                recall                     0.661      0.013   
                                f1_weighted                0.653      0.012   
                                roc_auc_ovr_weighted       0.868      0.018   
Logistic Regression    0.072    accuracy                   0.809      0.025   
                                balanced_accuracy          0.717      0.044   
                                precision                  0.811      0.029   
                                recall                     0.809      0.025   
                                f1_weighted                0.789      0.035   
                                roc_auc_ovr_weighted       0.964      0.004   
Support Vector Machine 0.019    accuracy                   0.717      0.017   
                                balanced_accuracy          0.648      0.022   
                                precision                  0.707      0.018   
                                recall                     0.717      0.017   
                                f1_weighted                0.705      0.019   
                                roc_auc_ovr_weighted       0.901      0.012   

                                                      Mean Test  Std Test  
Classifier             Fit Time Metric                                     
Random Forest          0.334    accuracy                  0.602     0.085  
                                balanced_accuracy         0.562     0.108  
                                precision                 0.601     0.089  
                                recall                    0.602     0.085  
                                f1_weighted               0.593     0.085  
                                roc_auc_ovr_weighted      0.820     0.047  
Decision Tree          0.010    accuracy                  0.402     0.028  
                                balanced_accuracy         0.342     0.026  
                                precision                 0.411     0.044  
                                recall                    0.402     0.028  
                                f1_weighted               0.391     0.029  
                                roc_auc_ovr_weighted      0.646     0.022  
Logistic Regression    0.072    accuracy                  0.602     0.072  
                                balanced_accuracy         0.508     0.071  
                                precision                 0.570     0.076  
                                recall                    0.602     0.072  
                                f1_weighted               0.573     0.064  
                                roc_auc_ovr_weighted      0.831     0.063  
Support Vector Machine 0.019    accuracy                  0.618     0.067  
                                balanced_accuracy         0.546     0.071  
                                precision                 0.605     0.067  
                                recall                    0.618     0.067  
                                f1_weighted               0.607     0.065  
                                roc_auc_ovr_weighted      0.837     0.057

In [11]:
import time

def model_eval(model, classifier_name, X_train, X_test, y_train, y_test):
    start_time = time.time()
    
    # Ensure that y_train and y_test are 1D arrays
    y_train = np.array(y_train).ravel()
    y_test = np.array(y_test).ravel()
    
    # Get predicted probabilities for ROC AUC
    pred_train_proba = model.predict_proba(X_train)
    pred_test_proba = model.predict_proba(X_test)
    
    # Get predicted classes for other metrics
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)
    
    # Calculate metrics
    metrics_train = {
        "accuracy": round(accuracy_score(y_train, pred_train), 3),
        "balanced_accuracy": round(balanced_accuracy_score(y_train, pred_train), 3),
        "precision": round(precision_score(y_train, pred_train, average="weighted"), 3),
        "recall": round(recall_score(y_train, pred_train, average="weighted"), 3),
        "f1_weighted": round(f1_score(y_train, pred_train, average="weighted"), 3),
        "roc_auc_ovr_weighted": round(roc_auc_score(y_train, pred_train_proba, average="weighted", multi_class="ovr"),3,),
    }
    
    metrics_test = {
        "accuracy": round(accuracy_score(y_test, pred_test), 3),
        "balanced_accuracy": round(balanced_accuracy_score(y_test, pred_test), 3),
        "precision": round(precision_score(y_test, pred_test, average="weighted"), 3),
        "recall": round(recall_score(y_test, pred_test, average="weighted"), 3),
        "f1_weighted": round(f1_score(y_test, pred_test, average="weighted"), 3),
        "roc_auc_ovr_weighted": round(roc_auc_score(y_test, pred_test_proba, average="weighted", multi_class="ovr"),3,),
    }
            
    elapsed_time = time.time() - start_time

    # Create the DataFrame without additional rounding issues
    pred_metrics_df = pd.DataFrame({
        "Classifier": classifier_name,
        "Classification Time": round(elapsed_time, 3),
        "Metric": list(metrics_train.keys()),
        "Train data": list(metrics_train.values()),
        "Test data": list(metrics_test.values()),
    })
    
    return pred_metrics_df

In [12]:
pred_rf = model_eval(model_rf,"Random Forest", X_train, X_test, y_train, y_test)
pred_dt = model_eval(model_dt,"Decision Tree", X_train, X_test, y_train, y_test)
pred_lg = model_eval(model_lg,"Logistic Regression", X_train_scaled, X_test_scaled, y_train, y_test)
pred_svm = model_eval(model_svm,"Support Vector Machine", X_train_scaled, X_test_scaled, y_train, y_test)

In [13]:
prediction_df = pd.concat([pred_rf, pred_dt, pred_lg, pred_svm])
prediction_df_report = prediction_df.set_index(["Classifier", "Classification Time", "Metric"])
prediction_df_report

Train data  \
Classifier             Classification Time Metric                             
Random Forest          0.083               accuracy                   0.926   
                                           balanced_accuracy          0.936   
                                           precision                  0.930   
                                           recall                     0.926   
                                           f1_weighted                0.925   
                                           roc_auc_ovr_weighted       0.990   
Decision Tree          0.031               accuracy                   0.675   
                                           balanced_accuracy          0.609   
                                           precision                  0.667   
                                           recall                     0.675   
                                           f1_weighted                0.660   
                                           roc_auc_ovr_weighted       0.874   
Logistic Regression    0.062               accuracy                   0.807   
                                           balanced_accuracy          0.715   
                                           precision                  0.807   
                                           recall                     0.807   
                                           f1_weighted                0.789   
                                           roc_auc_ovr_weighted       0.960   
Support Vector Machine 0.022               accuracy                   0.711   
                                           balanced_accuracy          0.636   
                                           precision                  0.698   
                                           recall                     0.711   
                                           f1_weighted                0.698   
                                           roc_auc_ovr_weighted       0.896   

                                                                 Test data  
Classifier             Classification Time Metric                           
Random Forest          0.083               accuracy                  0.597  
                                           balanced_accuracy         0.594  
                                           precision                 0.587  
                                           recall                    0.597  
                                           f1_weighted               0.577  
                                           roc_auc_ovr_weighted      0.802  
Decision Tree          0.031               accuracy                  0.507  
                                           balanced_accuracy         0.484  
                                           precision                 0.506  
                                           recall                    0.507  
                                           f1_weighted               0.485  
                                           roc_auc_ovr_weighted      0.722  
Logistic Regression    0.062               accuracy                  0.582  
                                           balanced_accuracy         0.549  
                                           precision                 0.606  
                                           recall                    0.582  
                                           f1_weighted               0.545  
                                           roc_auc_ovr_weighted      0.798  
Support Vector Machine 0.022               accuracy                  0.597  
                                           balanced_accuracy         0.595  
                                           precision                 0.596  
                                           recall                    0.597  
                                           f1_weighted               0.583  
                                           roc_auc_ovr_weighted      0.822